In [3]:
from scipy import spatial
def _cosine(a,b):
    ans = (1 - spatial.distance.cosine(a,b))
    return ans

In [4]:
from sentence_transformers import SentenceTransformer
def bertembedding(df):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    list_data = []
    for column in df:
        list_data+=df[column].tolist()

    sentences = list_data
    embeddings = model.encode(sentences)
    
    return embeddings

In [5]:
def DP_matching(set_rep_l,set_p_rep_l,window_size):
#set1:　　　　　　　set_rep_l
#set2:             set_p_rep_l
    set1_result = []
    set2_result = []
    set1_stop_len = 0
    set2_stop_len = 0 #紀錄當前走多少
    finish = max(len(set_rep_l),len(set_p_rep_l))
    set1_finish =  finish
    set2_finish = finish#終止條件

    set1_index = 0
    set2_index = 0
    while set1_stop_len<set1_finish and  set2_stop_len<set2_finish:
        temp = []  
        #print("set1_index:",set1_index)
        #print("set2_index:",set2_index)
        try: #0
            temp.append(_cosine(set_rep_l[set1_index],set_p_rep_l[set2_index]))
            #temp.append(1 - spatial.distance.cosine(set_rep_l[set1_index], set_p_rep_l[set2_index]))
        except:
            temp.append(0)
        for size in range(1,window_size):
            try: #1
                temp.append(_cosine(set_rep_l[set1_index+size],set_p_rep_l[set2_index]))
                #temp.append(1 - spatial.distance.cosine(set_rep_l[set1_index+1], set_p_rep_l[set2_index]))
            except:
                temp.append(0)

        for size in range(1,window_size):          
            try: #3
                temp.append(_cosine(set_rep_l[set1_index],set_p_rep_l[set2_index+size]))
                #temp.append(1 - spatial.distance.cosine(set_rep_l[set1_index], set_p_rep_l[set2_index+1]))
            except:
                temp.append(0) 
              
                
        #print(temp)
        #     A'    B'    C'  D'   E'(set1)
        #  A  0     1     2   ...  w
        #  B  w+1   w'+1  1'  ...  w'
        #  C  w+2   w'+2  0'' ...  w'' 
        #  D  w+3   w'+3  3''
        #  E              
        #(set2)   
        #print(temp)
        max_value = max(temp)
        max_index = temp.index(max_value)
        #print('max_index:',max_index)
        if temp[0] >0.95:
            if set1_index < len(set_rep_l):
                    set1_result.append(set1_index)
            else:
                set1_result.append(-1)
            if set2_index < len(set_p_rep_l):
                set2_result.append(set2_index)
            else:
                set2_result.append(-1)    
            set1_index +=1
            set2_index +=1
            set1_stop_len +=1
            set2_stop_len +=1
            continue       
        if max_index == 0:
                if set1_index < len(set_rep_l):
                    set1_result.append(set1_index)
                else:
                    set1_result.append(-1)
                if set2_index < len(set_p_rep_l):
                    set2_result.append(set2_index)
                else:
                    set2_result.append(-1)    
                set1_index +=1
                set2_index +=1
                set1_stop_len +=1
                set2_stop_len +=1
        elif max_index<window_size:    
            compare_cosine = []
            for size in range(1,window_size):
                try: #1
                    C1 = _cosine(set_rep_l[set1_index+max_index],set_p_rep_l[set2_index+size])
                    #C1 = 1 - spatial.distance.cosine(set_rep_l[set1_index+1], set_p_rep_l[set2_index+1])
                except:
                    C1 = 0
                compare_cosine.append(C1)

            #print(' C1:', C1,'\nC2:', C2)
            if all( max_value >= checkvalue  for checkvalue in compare_cosine)and max_value>=0.5:
                for num in range(max_index):
                     set2_result.append(-1)


                if set1_index < finish:
#                         set1_result.append(set1_index)
                    for num in range(max_index+1):
                         set1_result.append(set1_index+num)               
                else:
                    set1_result.append(-1)


                if set2_index <finish:
                    set2_result.append(set2_index)
                else:
                    set2_result.append(-1)   
                set1_index +=(1+max_index)
                set2_index +=1
                set1_stop_len +=(1+max_index)
                set2_stop_len +=1
                set1_finish += (1+max_index-1)                  
            else:
                set1_result.append(set1_index)
                set2_result.append(-1)
                set1_index +=1
        elif max_index>=window_size:
            compare_cosine = []
            for size in range(1,window_size):
                try: #1
                    C1 = _cosine(set_rep_l[set1_index+size],set_p_rep_l[set2_index+max_index])
                    #C1 = 1 - spatial.distance.cosine(set_rep_l[set1_index+1], set_p_rep_l[set2_index+1])
                except:
                    C1 = 0
                compare_cosine.append(C1)

            #print(' C1:', C1,'\nC2:', C2)
            if all( max_value >= checkvalue  for checkvalue in compare_cosine)and max_value>=0.5:
                for num in range(max_index-window_size+1):
                     set1_result.append(-1)

                #print("長度:",len(set_rep_l),"set2index:",set2_index)
                if set2_index < finish:
#                         set1_result.append(set1_index)
                    for num in range(max_index+1-window_size+1):
                         #print("set2 add :", set2_index+num)
                         set2_result.append(set2_index+num)               
                else:
                    set2_result.append(-1)


                if set1_index < finish:
                    set1_result.append(set1_index)
                else:
                    set1_result.append(-1) 
                    
                set1_index +=1
                set2_index +=(1+max_index-window_size+1)
                set1_stop_len +=1
                set2_stop_len +=(1+max_index-window_size+1)
                set2_finish += (1+max_index-1-window_size+1)                  
            else:
                set2_result.append(set2_index)
                set1_result.append(-1)
                set2_index +=1
        #print(set1_result)
        #print(set2_result)
        """
        print('stop1:',set1_stop_len)
        print('stop2:',set2_stop_len)
        print('finish1',set1_finish)
        print('finish2',set2_finish)
        print(set1_result)
        print(set2_result)
        """   
    set1_result = [x+1 for x in set1_result]
    set2_result = [x+1 for x in set2_result]
#     print(set1_result)
#     print(set2_result)
    while set1_result[-1] == 0 and set2_result[-1]==0:
        set1_result.pop()
        set2_result.pop()
    return  set1_result, set2_result


In [6]:
def set_embedding(serialnumber):
    train = col_temp.find_one({'serialNumber': serialnumber}) #weir Exalg
    set_rep = []
    #print(page)
    #start 處理 train data
    count_col = 0
    for sets_train in train['setsData']:       
        arr_t = np.array(sets_train)
        df = pd.DataFrame(arr_t, columns=['col'+str(item) for item in range(0,len(arr_t.T))])  
        row_count = 0
        data_train = []
        for i, row in df.iterrows():
            data_temp = ""
            for j, column in row.iteritems():
                data_temp = data_temp + column
            data_train.append(data_temp)
            row_count+=1
#         if row_count <= 3:
#             print("delete")
#             continue;
            
            
        d = {'text': data_train}
        row_df = pd.DataFrame(data=d)
        row_embedding = bertembedding(row_df)        
        row_embedding_np = np.matrix(row_embedding)
        avg_row_embedding = row_embedding_np.sum(axis=0)/len(row_embedding_np)
        set_rep.append(avg_row_embedding)
        count_col +=1
    return set_rep,train,count_col

In [7]:
def set_matching_api(master,slave):
    set_rep = []
    #print(page)
    #start 處理 train data
    set_rep,train,count_col_m = set_embedding(master)
    #for serialNumber in range(2,len(page)): #dacde failed
    set_p_rep,predict,count_col_s = set_embedding(slave)
    set_rep_l = []
    set_p_rep_l = []
    
    for x in set_rep:
        set_rep_l+=x.tolist()
    for x in set_p_rep:
        set_p_rep_l+=x.tolist()
    #print(page[serialNumber])
    #print('set_rep_l:',len(set_rep_l),'set_p_rep_l:',len(set_p_rep_l))
    #print(set_rep_l)
    
    set_d = len(train['setsData']) - len(predict['setsData'])
    #print(abs(set_d))
    window_size = abs(set_d) + 5
    print("len(train['setsData']):",len(train['setsData']))
    print("len(predict['setsData']:",len(predict['setsData']))
    print("window_size:", window_size)
    
    print("count_col_m:",count_col_m)
    print("count_col_s:",count_col_s)
    set_d2 = count_col_m - count_col_s
    window_size2 = abs(set_d2) + 5
    print("window_size2:", window_size2)
    
    if count_col_m ==0 and count_col_s ==0:
        dict ={}
        return dict
    else:
        set1_result,set2_result=DP_matching(set_rep_l,set_p_rep_l,window_size2)
#         print(page[serialNumber], file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
#         print(set1_result, file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
#         print(set2_result, file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
    """
    """
    dict ={} #用來處理多Page一起合併的結果
    first_merge = True #用來判斷是否為第一次合併
    collect_no_match_vec = [] #用來收集對齊到空的那些set的向量表示
    collect_no_match_num = [] #用來表示每個set有幾個對齊到空的數量 , for example, [2,3,4] 表示page1 有兩個set對到空 page2有3個...
    
    count_page1_no_match = 0
    count_page2_no_match = 0
    for i in range(len(set2_result)):
        if first_merge:
            if set1_result[i] in dict:
                #print(set1_result[i])
                dict.get(set1_result[i]).append(set2_result[i])
            else:
                temp = [set2_result[i]]            
                dict[set1_result[i]] = temp
        else:
            if set1_result[i] in dict:
                #print(set1_result[i])
                dict.get(set1_result[i]).append(set2_result[i])

        if set1_result[i] == 0: #表示page2的set2_result[i] 對齊到空集 要將資料額外取出來 最後再統整時重算相似度
                collect_no_match_vec.append(set_p_rep_l[set2_result[i]-1]) #page2的set2_result[i] 的向量表示, set2_result[i]表示在page2第i個set
                count_page2_no_match +=1

    collect_no_match_num.append(count_page2_no_match)
    """
    """
    
    
    print("testing",master,"and set1_result:",set1_result)
    print("testing",slave,"and set2_result:",set2_result)
    return dict
  
        #print("testing",page[serialNumber],"and set1_result:",set1_result)
 

In [8]:
def set_embedding_forDB(train):
    set_rep = []
    #print(page)
    #start 處理 train data
    train_index = [] # 表示在資料庫形式中，資料儲存的位置(index)
    train_i = 0
    count_col = 0
    for sets_train in train['setsData']:       
        arr_t = np.array(sets_train)
        df = pd.DataFrame(arr_t, columns=['col'+str(item) for item in range(0,len(arr_t.T))])  
        row_count = 0
        data_train = []
        for i, row in df.iterrows():
            data_temp = ""
            for j, column in row.iteritems():
                data_temp = data_temp + column
            data_train.append(data_temp)
            row_count+=1
        if row_count <= 3:
            train_i +=1 
            print("delete")
            continue;
        train_index.append(train_i)
        train_i +=1 
        d = {'text': data_train}
        row_df = pd.DataFrame(data=d)
        row_embedding = bertembedding(row_df)        
        row_embedding_np = np.matrix(row_embedding)
        avg_row_embedding = row_embedding_np.sum(axis=0)/len(row_embedding_np)
        set_rep.append(avg_row_embedding)
        count_col +=1
    return set_rep,train,count_col,train_index

In [9]:
def set_matching_api_forDB(master,slave):
    set_rep = []
    #print(page)
    #start 處理 train data
    set_rep,train,count_col_m,train_index = set_embedding_forDB(master)
    #for serialNumber in range(2,len(page)): #dacde failed
    set_p_rep,predict,count_col_s,predict_index = set_embedding_forDB(slave)
    set_rep_l = []
    set_p_rep_l = []
    
    for x in set_rep:
        set_rep_l+=x.tolist()
    for x in set_p_rep:
        set_p_rep_l+=x.tolist()
    #print(page[serialNumber])
    #print('set_rep_l:',len(set_rep_l),'set_p_rep_l:',len(set_p_rep_l))
    #print(set_rep_l)
    
    set_d = len(train['setsData']) - len(predict['setsData'])
    #print(abs(set_d))
    window_size = abs(set_d) + 5
    #print("len(train['setsData']):",len(train['setsData']))
    #print("len(predict['setsData']:",len(predict['setsData']))
    #print("window_size:", window_size)
    
    #print("count_col_m:",count_col_m)
    #print("count_col_s:",count_col_s)
    set_d2 = count_col_m - count_col_s
    window_size2 = abs(set_d2) + 5
    #print("window_size2:", window_size2)
    
    if count_col_m ==0 and count_col_s ==0:
        dict ={}
        return dict
    else:
        set1_result,set2_result=DP_matching(set_rep_l,set_p_rep_l,window_size2)
#         print(page[serialNumber], file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
#         print(set1_result, file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
#         print(set2_result, file=open("./schema_matching_data/set_result/Exalg/testebat"+str(countpage)+".txt", "a"))
    """
    """
    dict ={} #用來處理多Page一起合併的結果
    first_merge = True #用來判斷是否為第一次合併
    collect_no_match_vec = [] #用來收集對齊到空的那些set的向量表示
    collect_no_match_num = [] #用來表示每個set有幾個對齊到空的數量 , for example, [2,3,4] 表示page1 有兩個set對到空 page2有3個...
    
    count_page1_no_match = 0
    count_page2_no_match = 0
    for i in range(len(set2_result)):
        if first_merge:
            if set1_result[i] in dict:
                #print(set1_result[i])
                dict.get(set1_result[i]).append(set2_result[i])
            else:
                temp = [set2_result[i]]            
                dict[set1_result[i]] = temp
        else:
            if set1_result[i] in dict:
                #print(set1_result[i])
                dict.get(set1_result[i]).append(set2_result[i])

        if set1_result[i] == 0: #表示page2的set2_result[i] 對齊到空集 要將資料額外取出來 最後再統整時重算相似度
                collect_no_match_vec.append(set_p_rep_l[set2_result[i]-1]) #page2的set2_result[i] 的向量表示, set2_result[i]表示在page2第i個set
                count_page2_no_match +=1

    collect_no_match_num.append(count_page2_no_match)
    """
    """
    
    
    #print("testing",master,"and set1_result:",set1_result)
    #print("testing",slave,"and set2_result:",set2_result)
    return dict, train_index, predict_index
  
        #print("testing",page[serialNumber],"and set1_result:",set1_result)
 

In [12]:
def col_matching_forDB(set_result,train,predcit,train_index , predict_index,model_select=2):
    #print(type(train))
    print(set_result)
    result = train.copy()
    #print(type(result))
    for master_index,slave_index in set_result.items():
            #print("master_index:",master_index,"slave_index:",slave_index)
            #print("serialNumber_index:",serialNumber_index-1)
            #print("slave_index:",slave_index[serialNumber_index-1])
            #print("start set matching")
            #print(slave_index[serialNumber_index-1]-1)
            if master_index ==0: 
                #no_matching_set += len(slave_index)
                #print("master_index == 0 skip")
                for le in range(len(slave_index)):
                    train.get('setsData').append(predict['setsData'][le])
                continue
            else:
                if (slave_index[0]-1) == -1:
                   # print("slave_index[0]-1 skip")
                    #no_matching_set += len(slave_index)
                    continue
                else:

                    sets_train = train['setsData'][train_index[master_index-1]]
                    #sets_test = predcit['setsData'][slave_index[serialNumber_index-1]-1]
                    if (slave_index[0]-1) <0:
                        print("slave_index[0]-1 <0 skip",slave_index[0])
                        continue;
                    sets_test = predcit['setsData'][predict_index[slave_index[0]-1]]

                    if len(sets_train[0]) == len(sets_test[0]):
                        #print()
                        print("same col")
                    else:
                        print("diff col",len(sets_train[0])," ",len(sets_test[0]))
                        #error += 1
                        continue;
                    arr_t = np.array(sets_train)

                    df = pd.DataFrame(arr_t, columns=['col'+str(item) for item in range(0,len(arr_t.T))])

                    arr_p = np.array(sets_test)

                    df_predict = pd.DataFrame(arr_p, columns=['col'+str(item) for item in range(0,len(arr_p.T))])    

                    embeddings = extract(df)

                    embeddings_predict = extract(df_predict)

                    # #df


                    label_ = []
                    for j in range(0,len(df.columns)):
                        for i in range(0,len(df)):
                            label_.append(j)

                    label_prdict = []
                    for j in range(0,len(df_predict.columns)):
                        for i in range(0,len(df_predict)):
                            label_prdict.append(j)                    

                    label_ = np.array(label_)
                    #print(len(label_))
                    label_prdict = np.array(label_prdict)
                    #print(len(label_prdict))




                    if model_select == 1:
                        if 1 in label_:
                            my_model = svm.SVC(probability=True) 
                            my_model = my_model.fit(embeddings, label_)
                    elif model_select == 2: 

                        knn_clf = neighbors.KNeighborsClassifier(n_neighbors = len(df.columns))
                        my_model = knn_clf.fit(embeddings, label_)
                        y_predicted = my_model.predict(embeddings_predict)
                        #print(y_predicted)
                        #print(label_prdict)
                    elif model_select == 3:
                        learning_rate = 0.003
                        #learning_rate = 0.001
                        epochs = 8
                        batch_size = 4
                        validation_split = 0.1
                        class_num = len(df.columns)
                        my_model = create_model(learning_rate,class_num)

                        # Train the model on the normalized training set.
                        epochs, hist = train_model(my_model, embeddings, label_, epochs, batch_size, validation_split)

                        # Plot a graph of the metric vs. epochs.
                        list_of_metrics_to_plot = ['accuracy']
                        #plot_curve(epochs, hist, list_of_metrics_to_plot)

                        # Evaluate against the test set.
                        #print("\n Evaluate the new model against the test set:") 
                        #loss, accuracy, f1_score, precision, recall = my_model.evaluate(x=embeddings_predict, y=label_prdict, batch_size=batch_size)
                        y_pred = my_model.predict(embeddings_predict)



                column_num = len(df.columns)
                testing_row_number = len(df_predict)
                if model_select == 3 :
                    group_vote = cal_vote_result_for_NN(my_model,column_num,testing_row_number,embeddings_predict)
                    ans = [x for x in range(0,column_num)]
                    column_result = classifier(column_num,group_vote)
                else:
                    if 1 in label_:
                        group_vote = cal_vote_result_for_KNN(my_model,column_num,testing_row_number,embeddings_predict)
                        ans = [x for x in range(0,column_num)]
                        column_result = classifier(column_num,group_vote)
                    else:
                        ans = [0]         
                        column_result = [0]
                #print(ans)
                #print(column_result)

                #res = sum(x == y for x, y in zip(ans, column_result)) #計算當前set 正確的col數
                #print(res)
                #count_all_col_num_temp += len(ans)
                #count_correct_num_temp += res
                #Evaluation(ans,column_result,f1,precision,recall,accuracy_e,ans_list,column_result_list)
                #ans_.append(x[serialNumber_index])
                #ans_index.append(serialNumber_index)

                #紀錄main set正確col數量
                #if count_current_set_index == main:
                #    main_count_all_col_num.append(len(ans)) 
                #    main_count_correct_num.append(res)
                #    main_count_error_num.append(len(ans)-res)

                #tmp = res/len(ans) #計算當前set準確度
                #evl_avg_set += tmp

                #count_current_set_index +=1
                print(column_result)
                #total_set_num +=1
                
                #print(column_result)
                #print(predict['setsData'][slave_index[0]-1])
                index = master_index-1
                #print(result['setsData'][index])
                for i in range(len(predict['setsData'][predict_index[slave_index[0]-1]])):
                    #print(i)
                    temp_arr = []
                    
                    for x in column_result:
                        #print(slave_index[0]-1)
                        #print(predict_index[slave_index[0]-1])
                        #print('x:',x)
                        #print('i:',i)
                        #print(len(predict['setsData'][predict_index[slave_index[0]-1]][i]))
                        #print(predict['setsData'][predict_index[slave_index[0]-1]][i])
                        #print(predict['setsData'][predict_index[slave_index[0]-1]][i][x])
                        #print(len(predict['setsData'][predict_index[slave_index[0]-1]][i][x]))
                        temp_arr.append(predict['setsData'][predict_index[slave_index[0]-1]][i][x])
                    
                    
                    #print(result['setsData'][index])
                    result['setsData'][train_index[index]].append(temp_arr)
                #print(result['setsData'][index])    
                print("done")
    #print(result)
    
    return result